# 第三章 迭代优化

当使用 LLM 构建应用程序时，实践层面上很难*第一次尝试*就成功获得适合最终应用的 Prompt。但这并不重要，只要您有一个<span style="color:red">好的迭代过程</span>来不断改进您的 Prompt，那么您就能够得到一个适合任务的 Prompt。虽然相比训练机器学习模型，在  Prompt 方面一次成功的几率可能会高一些，但正如上所说， Prompt 是否一次完善并不重要。最重要的是<span style="color:red">层层迭代</span>为您的应用程序找到有效  Prompt 的过程。

因此在本章中，我们将以产品说明书中生成营销文案为例，来展示一些流程框架，并提示您思考如何层层迭代地分析和完善您的 Prompt。

在吴恩达（Andrew Ng，原教程作者）的机器学习课程中展示过一张图表，说明了机器学习开发的流程。通常是先有一个想法，然后再用以下流程实现：编写代码，获取数据，训练模型，获得实验结果。然后您可以查看结果，分析误差与错误，找出适用领域，甚至可以更改您对具体问题的具体思路或解决方法。此后再次更改实现，并运行另一个实验等，反复迭代，最终获得有效的机器学习模型。在编写基于 LLM 的应用程序的 Prompt 时，流程可能非常相似。您产生了关于要完成的任务的想法后，可以尝试编写第一个 Prompt ，注意要满足上一章说过的两个原则：**清晰明确，并且给系统足够的时间思考**。然后您可以运行并查看结果。如果第一次效果不好，那么迭代的过程就是找出为什么指令不够清晰或为什么没有给算法足够的时间思考，以便改进想法、改进  Prompt 等等，循环多次，直到找到适合您的应用程序的 Prompt。

很难有适用于世间万物的所谓“最佳  Prompt ”，更好的方法是找到<span style="color:red">有效的迭代过程</span>，以便您可以快速地找到一个适合您的应用程序的  Prompt 。


<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一环境配置" data-toc-modified-id="一、环境配置">一、环境配置</a></span></li>
        <li>
            <span><a href="#二任务从产品说明书生成一份营销产品描述" data-toc-modified-id="二、任务——从产品说明书生成一份营销产品描述">二、任务——从产品说明书生成一份营销产品描述</a></span>
            <ul class="toc-item">
                <li><span><a href="#21-问题一生成文本太长" data-toc-modified-id="2.1 问题一：生成文本太长">2.1 问题一：生成文本太长</a></span></li>
                <li><span><a href="#22-问题二抓错文本细节" data-toc-modified-id="2.2 问题二：抓错文本细节">2.2 问题二：抓错文本细节</a></span></li>
                <li><span><a href="#23-问题三添加表格描述" data-toc-modified-id="2.3 问题三：添加表格描述">2.3 问题三：添加表格描述</a></span></li>
            </ul>
        </li>
    </ul>
</div>

## 一、环境配置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

In [3]:
client = OpenAI(api_key=openai_api_key)

def get_completion(prompt, model="gpt-4o-mini"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为gpt-4o-mini
    '''
    # 调用API
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    # 返回结果
    return response.choices[0].message.content

## 二、任务——从产品说明书生成一份营销产品描述

给定一份椅子的资料页。描述说它属于*中世纪灵感*系列，产自意大利，并介绍了材料、构造、尺寸、可选配件等参数。假设您想要使用这份说明书帮助营销团队为电商平台撰写营销描述稿：

In [4]:
# 示例：产品说明书
fact_sheet_chair = """
概述

    美丽的中世纪风格办公家具系列的一部分，包括文件柜、办公桌、书柜、会议桌等。
    多种外壳颜色和底座涂层可选。
    可选塑料前后靠背装饰（SWC-100）或10种面料和6种皮革的全面装饰（SWC-110）。
    底座涂层选项为：不锈钢、哑光黑色、光泽白色或铬。
    椅子可带或不带扶手。
    适用于家庭或商业场所。
    符合合同使用资格。

结构

    五个轮子的塑料涂层铝底座。
    气动椅子调节，方便升降。

尺寸

    宽度53厘米|20.87英寸
    深度51厘米|20.08英寸
    高度80厘米|31.50英寸
    座椅高度44厘米|17.32英寸
    座椅深度41厘米|16.14英寸

选项

    软地板或硬地板滚轮选项。
    两种座椅泡沫密度可选：中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）。
    无扶手或8个位置PU扶手。

材料
外壳底座滑动件

    改性尼龙PA6/PA66涂层的铸铝。
    外壳厚度：10毫米。
    座椅
    HD36泡沫

原产国

    意大利
"""

In [5]:
#   Prompt ：基于说明书创建营销描述
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的营销描述。

根据下文中由三个尖括号给出的技术说明书中提供的信息，编写一个营销描述。

技术说明: <<<{fact_sheet_chair}>>>
"""
response = get_completion(prompt)
print(response)

**产品营销描述：中世纪风格办公家具系列**

探索我们精美的中世纪风格办公家具系列，完美融合了经典设计与现代功能。无论是在家庭办公室还是商业环境中，这款系列都能为您的工作空间增添一抹优雅与舒适。

我们的办公家具包括文件柜、办公桌、书柜和会议桌等多种选择，您可以根据个人喜好选择多种外壳颜色和底座涂层，确保每一件家具都与您的空间完美契合。您还可以选择塑料前后靠背装饰（SWC-100）或10种面料和6种皮革的全面装饰（SWC-110），为您的办公环境增添个性化的风格。

每把椅子都配备五个轮子的塑料涂层铝底座，确保灵活移动，同时气动调节功能让您轻松调整座椅高度，适应不同的工作需求。无论您选择带扶手还是无扶手的设计，我们的椅子都提供中等（1.8磅/立方英尺）或高（2.8磅/立方英尺）两种座椅泡沫密度选项，确保您在长时间工作时依然感到舒适。

我们的办公家具不仅符合合同使用资格，还提供软地板或硬地板滚轮选项，适应各种办公环境。每一件家具均在意大利精心制造，展现出卓越的工艺与品质。

选择我们的中世纪风格办公家具系列，让您的工作空间焕发出独特的魅力与舒适感，提升您的工作效率与生活品质。立即体验这一经典与现代完美结合的办公家具系列！


## 2.1 问题一：生成文本太长

它似乎很好地完成了要求，即从技术说明书开始编写产品描述，介绍了一个精致的中世纪风格办公椅。但是当我看到这个时，我会觉得这个太长了。

所以在上述过程中，我产生想法后写了一个  Prompt ，并得到了结果，但是我对它不是很满意，因为它太长了。所以我澄清我的  Prompt ，要求它限制生成文本长度，要求最多使用50个字。


提取回答并根据空格拆分，答案为54个字，较好地完成了设计要求。

In [7]:
# 优化后的 Prompt，要求生成描述不多于 50 词
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的营销描述。

根据下文中由三个尖括号给出的技术说明书中提供的信息，编写一个营销描述。

使用最多50个汉字。

技术说明: <<<{fact_sheet_chair}>>>
"""
response = get_completion(prompt)
print(response)

中世纪美学办公椅，意大利匠心打造。多色外壳/涂层可选，丰富面料皮革饰面。气动升降+五轮底座，软硬地板适配。高密度泡沫座椅，8向扶手调节，家用商馆皆宜。优雅与实用并存，重塑办公舒适体验。


In [8]:
# 由于中文需要分词，此处直接计算整体长度
len(response)

92

LLM在能堪堪胜任严格的字数限制，但实现得并不精确。此例中，英文输出要求控制在50个词，但有时会输出60或65个单词的内容，但这也还算合理。原因是 LLM 使用分词器（tokenizer）解释文本，但它们往往在计算字符方面表现一般般。有很多不同的方法来尝试控制您得到的输出的长度（如若干句话/词/个汉字/个字母 (characters) 等）。

## 2.2 问题二：抓错文本细节

我们继续完善这段推广词，会发现的第二个问题是，这个网站并不是直接向消费者销售，它实际上面向的是家具零售商，他们会更关心椅子的技术细节和材料。在这种情况下，您可以继续修改这个  Prompt ，让它更精确地描述椅子的技术细节。

解决方法：要求它专注于与目标受众相关的方面。

In [10]:
# 优化后的 Prompt，说明面向对象，应具有什么性质且侧重于什么方面
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据下文中由三个尖括号给出的技术说明书中提供的信息，编写一个营销描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

使用最多50个汉字。

技术说明: <<<{fact_sheet_chair}>>>
"""
response = get_completion(prompt)
print(response)

中世纪风格办公椅，改性尼龙铸铝外壳（10mm），五轮塑料涂层铝基可选不锈钢/哑光黑/白/铬。气动升降，HD36泡沫座垫（1.8/2.8磅密度），PU八向扶手，意大利制造，符合商用标准。（50字）


可见，通过修改  Prompt ，模型的关注点倾向了具体特征与技术细节。

我可能进一步想要在描述的结尾展示出产品ID。因此，我可以进一步改进这个  Prompt ，要求在描述的结尾，展示出说明书中的7位产品ID。

In [11]:
# 更进一步
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据下文中由三个尖括号给出的技术说明书中提供的信息，编写一个营销描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

使用最多50个汉字。

技术说明: <<<{fact_sheet_chair}>>>
"""
response = get_completion(prompt)
print(response)

中世纪风格办公椅，铸铝改性尼龙底座搭配五轮结构，气动升降系统。可选10mm厚PA6/PA66外壳与HD36泡沫座椅，提供两种装饰方案（SWC-100塑料/SWC-110全包覆）。底座涂层四选，含不锈钢/铬选项，意大利制造。产品ID：SWC-100/SWC-110


以上是许多开发人员通常会经历的  Prompt 开发的迭代过程简短示例。我的建议是，像上一章中所演示的那样，Prompt 应该保持清晰和明确，并在必要时给模型一些思考时间。在这些要求的基础上，常见流程是首先尝试编写一版 Prompt ，看看会发生什么，然后继续迭代完善 Prompt，以逐渐接近所需的结果。许多成功的 Prompt 都是通过这种迭代过程得出的。我将向您展示一个更复杂的 Prompt 示例，可能会让您对 ChatGPT 的能力有更深入的了解。

## 2.3 问题三：添加表格描述
继续添加指引，要求提取产品尺寸信息并组织成表格，并指定表格的列、表名和格式；再将所有内容格式化为可以在网页使用的 HTML。

In [15]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

在描述之后，额外生成一个表格，提供产品的尺寸。表格应该有两列。第一列包括尺寸的名称。第二列只包括英寸的测量值。

给表格命名为“产品尺寸”。

将所有内容格式化为可用于网站的HTML格式。将描述放在<div>元素中。

除了制定的要求，不要额外生成其他内容。

技术规格：```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
    <h2>中世纪风格办公椅技术说明</h2>
    <p>本产品为专业级办公家具系列的核心部件，采用铸铝基底结构，配以改性尼龙PA6/PA66涂层底座滑动件（厚度10毫米）。五爪式塑料涂层铝制底座配备双模滚轮系统，支持软/硬地面自适应。升降机构采用工业级气动调节装置，实现20cm行程范围内的精确高度控制。</p>
    
    <h3>材料配置</h3>
    <ul>
        <li>外壳结构：高压铸造铝框架，PA6/PA66复合涂层</li>
        <li>座椅系统：HD36高密度泡沫芯（可选1.8/2.8 lb/ft³密度等级）</li>
        <li>表面处理：模块化装饰系统（SWC-100型塑料背板/SWC-110型全包覆式面套）</li>
        <li>金属部件：可选不锈钢/哑光黑/亮白/镀铬四种PVD涂层</li>
    </ul>

    <h3>工程特性</h3>
    <ul>
        <li>符合ANSI/BIFMA商业家具标准</li>
        <li>多关节PU扶手系统（8向调节/可选拆除配置）</li>
        <li>双密度尼龙万向轮（动态载荷能力300kg）</li>
        <li>意大利原厂装配生产</li>
    </ul>

    <p>产品编号：SWC-100 / SWC-110</p>
</div>

<table>
    <caption>产品尺寸</caption>
    <tr>
        <th>尺寸名称</th>
        <th>英寸测量值</th>
    </tr>
    <tr><td>宽度</td><td>20.87</td></tr>
    <tr><td>深度</td><td>20.08</td></tr>
    <tr><td>高度</td><td>31.50</td></tr>
    <tr><td>座椅高度</td><td>17.32</td></tr>
    <tr><td>座椅深度</td><td>16.14</td></tr>
</table>


In [16]:
# 表格是以 HTML 格式呈现的，加载出来
from IPython.display import display, HTML

display(HTML(response))

尺寸名称,英寸测量值
宽度,20.87
深度,20.08
高度,31.50
座椅高度,17.32
座椅深度,16.14


<span style="color:#00ff00">
发现生成的描述中包含原本技术说明书中没有的内容（如荷载能力），进一步迭代prompt
</span>

In [19]:
# 要求它抽取信息并组织成表格，并指定表格的列、表名和格式
prompt = f"""
您的任务是帮助营销团队基于技术说明书创建一个产品的零售网站描述。

根据```标记的技术说明书中提供的信息，编写一个产品描述。

该描述面向家具零售商，因此应具有技术性质，并侧重于产品的材料构造。

在描述末尾，包括技术规格中每个7个字符的产品ID。

在描述之后，额外生成一个表格，提供产品的尺寸。表格应该有两列。第一列包括尺寸的名称。第二列只包括英寸的测量值。

给表格命名为“产品尺寸”。

将所有内容格式化为可用于网站的HTML格式。将描述放在<div>元素中。

请注意以下限制：

1.除了制定的要求，不要额外生成其他内容。

2.所有内容必须完全来自提供的技术说明书中，不要自行添加额外信息。

3.生成的内容尽量用中文描述。

技术规格：```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <p>本中世纪风格办公椅采用改性尼龙PA6/PA66涂层的铸铝底座结构，配备五个塑料包覆轮组，支持软/硬地板滚轮选项。气动升降系统提供便捷的高度调节功能，底座可选不锈钢/哑光黑/亮白/镀铬四种涂层。</p>

  <p>外壳使用10毫米厚度改性尼龙材质，座椅采用HD36高密度泡沫，提供1.8lb/ft³和2.8lb/ft³两种密度选择。装饰系统支持SWC-100型号的塑料靠背板或SWC-110型号的全包覆式面料/皮革装饰（10种布艺+6种皮革选项）。扶手配置包含无扶手或8向调节PU材质扶手两种方案。</p>

  <p>产品符合商业场所使用标准，结构组件通过意大利原厂认证。适用于家庭办公和商业环境，提供多种配色组合方案。</p>

  <p>产品ID：SWC-100, SWC-110</p>
</div>

<table>
  <caption>产品尺寸</caption>
  <tr>
    <td>宽度</td>
    <td>20.87</td>
  </tr>
  <tr>
    <td>深度</td>
    <td>20.08</td>
  </tr>
  <tr>
    <td>高度</td>
    <td>31.50</td>
  </tr>
  <tr>
    <td>座椅高度</td>
    <td>17.32</td>
  </tr>
  <tr>
    <td>座椅深度</td>
    <td>16.14</td>
  </tr>
</table>


In [20]:
display(HTML(response))

宽度,20.87
深度,20.08
高度,31.50
座椅高度,17.32
座椅深度,16.14


本章的主要内容是 LLM 在开发应用程序中的迭代式  Prompt 开发过程。开发者需要先尝试编写  Prompt ，然后通过迭代逐步完善它，直至得到需要的结果。作为一名高效的提示词工程师（Prompt Engineer），关键在于掌握<span style="color:red">有效的开发Prompt的过程</span>，而不是去寻求得到“完美的”Prompt。对于一些更复杂的应用程序，可以对多个样本（如数百张说明书）进行  Prompt 的迭代开发，并在样本集上进行评估。

最后，在更成熟的应用程序中，可以观察多个Prompt在多个样本集上的表现，测试平均或最差性能。但通常，**仅当**应用较成型之后，才推荐您通过这种评估方式，来精益求精。

请使用 Jupyter Notebook，动手实践本节给出的示例，并尝试不同的变化，查看结果。